In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
# 이미지 데이터의 경로 설정
data_root = "C:/Users/sunghoon/DL_python/chromate/"
image_folder_names = ['NG', 'OK']  # NG, OK 이미지 폴더 이름

# 이미지 전처리 및 데이터 로드
transform = transforms.Compose([
    transforms.Resize((64, 64)),   # 이미지 크기 조정
    transforms.ToTensor(),         # 이미지를 Tensor로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지 정규화
])

In [7]:
# CustomDataset 정의
class CustomDataset(Dataset):
    def __init__(self, data_root, transform=None):
        self.data_root = data_root
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, folder_name in enumerate(image_folder_names):
            folder_path = os.path.join(data_root, folder_name)
            image_names = [img_name for img_name in os.listdir(folder_path) if not img_name.startswith('.')]
            self.image_paths.extend([os.path.join(folder_path, img_name) for img_name in image_names])
            self.labels.extend([label] * len(image_names))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
# CustomDataset 인스턴스 생성
dataset = CustomDataset(data_root, transform=transform)

In [9]:
# 데이터를 train과 test로 분할
train_size = 0.8
train_dataset, test_dataset = train_test_split(dataset, train_size=train_size, shuffle=True)


In [10]:
# DataLoader 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [11]:
# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [12]:
# 모델 생성 및 손실 함수, 최적화기 정의
num_classes = len(image_folder_names)
model = CNNModel(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.6742
Epoch [2/10], Loss: 0.5771
Epoch [3/10], Loss: 0.2313
Epoch [4/10], Loss: 0.2114
Epoch [5/10], Loss: 0.1637
Epoch [6/10], Loss: 0.0631
Epoch [7/10], Loss: 0.0531
Epoch [8/10], Loss: 0.0235
Epoch [9/10], Loss: 0.0168
Epoch [10/10], Loss: 0.0039


In [14]:
# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test images: {(correct / total) * 100:.2f}%")


Accuracy on test images: 100.00%
